In [1]:
import pandas as pd
import numpy as np

from underthesea import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

# from keras.layers import merge
from keras.layers import Concatenate
from keras.layers.core import Dense, Reshape
from keras.layers import Embedding
from keras.models import Sequential
from scipy import spatial

# a)	Sử dụng độ đo Bleu score

In [22]:
import os
dataPath = os.path.dirname(os.getcwd())
print(dataPath)
filename1 = os.path.join(dataPath, 'data', 'chatbot-data.csv')
filename2 = os.path.join(dataPath, 'data', 'chatbot-data-2.csv')
filename1

e:\Workspace\NodeJS\chatbot-machine-learning-nodejs


'e:\\Workspace\\NodeJS\\chatbot-machine-learning-nodejs\\data\\chatbot-data.csv'

In [23]:
# filename1 = './data/chatbot data.csv'
# filename2 = './data/chatbot data 2.csv'
combined_csv = pd.concat([pd.read_csv(f) for f in [filename1, filename2]])
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8')
df = pd.read_csv("combined_csv.csv")
print(df.head(5))
print(df.shape)
# (7380, 3)

   Unnamed: 0                                             user_a  \
0           0                                        bạn tên gì?   
1           1                   Tui tên Nam <eos> Bạn học ở đâu?   
2           2         tui học ở TDTU luôn <eos> Bạn học ngành gì   
3           3                          Chuyên ngành gì vậy bạn ?   
4           4  mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...   

                                              user_b  
0             Mình tên Trường <eos> còn bạn tên gì?   
1  tui học ở TDTU nè <eos> còn bạn học ở trường n...  
2                       mình học Công Nghệ Thông Tin  
3  mình học bên Kỹ Thuật Phần Mềm á <eos> còn bạn...  
4  Mình là người Đồng tháp <eos> bạn biết Đồng Th...  
(7380, 3)


In [14]:
# Punctuation Removal
import string

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
df['clean_user_a']= df['user_a'].apply(lambda x:remove_punctuation(x))
df['user_a_lower']= df['clean_user_a'].apply(lambda x: x.lower())
df[['user_a','clean_user_a','user_a_lower']]

,user_a,clean_user_a,user_a_lower
0,bạn tên gì?,bạn tên gì,bạn tên gì
1,Tui tên Nam <eos> Bạn học ở đâu?,Tui tên Nam eos Bạn học ở đâu,tui tên nam eos bạn học ở đâu
2,tui học ở TDTU luôn <eos> Bạn học ngành gì,tui học ở TDTU luôn eos Bạn học ngành gì,tui học ở tdtu luôn eos bạn học ngành gì
3,Chuyên ngành gì vậy bạn ?,Chuyên ngành gì vậy bạn,chuyên ngành gì vậy bạn
4,mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...,mình học Quản Trị Kinh Doanh eos Bạn là người ...,mình học quản trị kinh doanh eos bạn là người ...
...,...,...,...
7375,Nhà mày bao nhiêu người?,Nhà mày bao nhiêu người,nhà mày bao nhiêu người
7376,Có bao nhiêu anh chị em?,Có bao nhiêu anh chị em,có bao nhiêu anh chị em
7377,Mày là em út hay anh cả?,Mày là em út hay anh cả,mày là em út hay anh cả
7378,Bố mày làm nghề gì?,Bố mày làm nghề gì,bố mày làm nghề gì


In [15]:
sentences = [word_tokenize(line) for line in df['user_a_lower']]
sentences[:10]

[['bạn', 'tên', 'gì'],
 ['tui', 'tên', 'nam', 'eos', 'bạn', 'học', 'ở', 'đâu'],
 ['tui', 'học', 'ở', 'tdtu', 'luôn', 'eos', 'bạn', 'học', 'ngành', 'gì'],
 ['chuyên', 'ngành', 'gì', 'vậy', 'bạn'],
 ['mình',
  'học',
  'quản trị',
  'kinh doanh',
  'eos',
  'bạn',
  'là',
  'người',
  'ở',
  'đâu',
  'vậy'],
 ['mình', 'biết', 'đồng', 'tháp', 'dưới', 'miền', 'tây', 'đúng', 'không'],
 ['còn',
  'mình',
  'ở',
  'tphcm',
  'luôn',
  'nè',
  'mình',
  'ở',
  'quận',
  '4',
  'eos',
  'cuối',
  'tuần',
  'này',
  'bạn',
  'có',
  'rãnh',
  'không'],
 ['mình', 'tính', 'cuối', 'tuần', 'này', 'rủ', 'bạn', 'đi', 'coffee', 'nè'],
 ['đi', 'phúc', 'long', 'đối diện', 'trường', 'mình', 'nè'],
 ['được',
  'luôn',
  'bạn',
  'sáng',
  'mình',
  'cũng',
  'rãnh',
  'eos',
  'bạn',
  'thích',
  'ngôn ngữ',
  'lập trình',
  'nào',
  'nhất']]

In [16]:
import warnings
warnings.filterwarnings('ignore')

def blue(candidate):
  max_bleu = 0
  max_id = 0
  for line in range(0,len(sentences)):
    t = sentence_bleu([sentences[line]], word_tokenize(candidate))
    if t > max_bleu:
      max_bleu = t
      max_id = line
  
  return max_id

candidate = 'Cuộc sống bạn ổn không'
max_id = blue(candidate)
print(candidate)
print(df['user_b'][max_id])

Cuộc sống bạn ổn không
Cũng ổn


In [ ]:
print("**************************")
print("*    start chatting      *")
print("*    input q to exit     *")
print("**************************")
while True:
  newMessage  = input("you : ")
  if newMessage == 'q':
    print("chatbot :","tạm biệt")
    break
  max_id = blue(newMessage)
  print("chatbot :",df['user_b'][max_id])

**************************
*    start chatting      *
*    input q to exit     *
**************************
you : đang làm gì đó?
chatbot : Bấm điện thoại
you : ăn trưa không
chatbot : ok luôn
you : ăn gì
chatbot : gì cũng được
you : ăn hủ tiếu không
chatbot : không
you : ăn phở nha
chatbot : Đi anh ơi
you : ăn ở đâu
chatbot : KFC chứ đâu
you : 15 phút nữa mình sang
chatbot : Ok
you : q
chatbot : tạm biệt


In [ ]:
# **************************
# *    start chatting      *
# *    input q to exit     *
# **************************
# you : đang làm gì đó?
# chatbot : Bấm điện thoại
# you : ăn trưa không
# chatbot : ok luôn
# you : ăn gì
# chatbot : gì cũng được
# you : ăn hủ tiếu không
# chatbot : không
# you : ăn phở nha
# chatbot : Đi anh ơi
# you : ăn ở đâu
# chatbot : KFC chứ đâu
# you : 15 phút nữa mình sang
# chatbot : Ok
# you : q
# chatbot : tạm biệt

# b)	Sử dụng Word2Vec và độ đo Euclid 

In [ ]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(sentences)
word2id = t.word_index
vocab_size = len(t.word_index) + 1
print(vocab_size)
print(t.index_word)

In [18]:
wids = [[word2id[w] for w in doc] for doc in sentences]
wids[:10]

[[3, 28, 4],
 [79, 28, 139, 18, 3, 16, 7, 8],
 [79, 16, 7, 531, 136, 18, 3, 16, 80, 4],
 [912, 80, 4, 19, 3],
 [15, 16, 1162, 913, 18, 3, 13, 38, 7, 8, 19],
 [15, 61, 532, 1163, 481, 264, 672, 174, 1],
 [56, 15, 7, 914, 136, 116, 15, 7, 180, 243, 18, 119, 117, 27, 3, 2, 181, 1],
 [15, 144, 119, 117, 27, 168, 3, 5, 1591, 116],
 [5, 915, 392, 772, 32, 15, 116],
 [25, 136, 3, 265, 15, 52, 181, 18, 3, 11, 533, 916, 9, 57]]

In [19]:
from keras.preprocessing.sequence import skipgrams

# generate skip-grams
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=2) for wid in wids]
skip_grams[0]

([[3, 4],
  [28, 4],
  [4, 28],
  [4, 3],
  [28, 656],
  [4, 1965],
  [3, 1387],
  [4, 444],
  [28, 548],
  [3, 2689],
  [28, 3],
  [3, 28]],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1])

In [20]:
pairs, labels = skip_grams[0][0], skip_grams[0][1]
print(sentences[0])
for i in range(15):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          t.index_word[pairs[i][0]], pairs[i][0], 
          t.index_word[pairs[i][1]], pairs[i][1], 
          labels[i]))

['bạn', 'tên', 'gì']
(bạn (3), gì (4)) -> 1
(tên (28), gì (4)) -> 1
(gì (4), tên (28)) -> 1
(gì (4), bạn (3)) -> 1
(tên (28), thể (656)) -> 0
(gì (4), out (1965)) -> 0
(bạn (3), cha (1387)) -> 0
(gì (4), đôi (444)) -> 0
(tên (28), hẹn hò (548)) -> 0
(bạn (3), công khai (2689)) -> 0
(tên (28), bạn (3)) -> 1
(bạn (3), tên (28)) -> 1


IndexError: list index out of range

In [22]:
from keras.layers import Input
from keras.models import Model
# from keras.layers.embeddings import Embedding
from keras.layers import Embedding
from keras.layers.core import Dense, Reshape
from keras.layers import dot

embed_size = 400
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, embed_size, input_length=1, name='embedding')

word_embedding = embedding(input_target)
word_embedding = Reshape((embed_size, 1))(word_embedding)
context_embedding = embedding(input_context)
context_embedding = Reshape((embed_size, 1))(context_embedding)
 
dot_product = dot([word_embedding, context_embedding], axes=1)
dot_product = Reshape((1,))(dot_product)

output = Dense(1, activation='sigmoid')(dot_product)

model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(loss='mean_squared_error', optimizer='rmsprop')

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 400)       1199200     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 400, 1)       0           ['embedding[0][0]']          

In [23]:
for epoch in range(0, 10):
    loss = 0
    for i, elem in enumerate(skip_grams):
        # print('i = ',i,'elem = ',elem)
        if elem[0] == []:
          continue
        pair_first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        pair_second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [pair_first_elem, pair_second_elem]
        Y = labels

        loss += model.train_on_batch(X,Y)  

    print('Epoch:', epoch, 'Loss:', loss)

Epoch: 0 Loss: 1260.8265806819254
Epoch: 1 Loss: 1147.2681394497267
Epoch: 2 Loss: 1115.0315442279025
Epoch: 3 Loss: 1103.3712853053803
Epoch: 4 Loss: 1098.3239084510642
Epoch: 5 Loss: 1095.83362436256
Epoch: 6 Loss: 1094.6279159171827
Epoch: 7 Loss: 1094.1569566896214
Epoch: 8 Loss: 1094.1562129269078
Epoch: 9 Loss: 1094.495574378525


In [26]:
T = model.layers[2]
weights = T.get_weights()[0][1:]

print(weights.shape)
df_embedding = pd.DataFrame(weights, index=t.index_word.values())
df_embedding

(2997, 400)


,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
không,-4.455145,0.478993,1.533078,-0.073596,0.455232,1.342729,0.478562,-2.815515,0.161068,0.600934,...,-0.445684,1.545106,1.041564,1.812963,-0.081663,-4.084830,2.517688,1.660002,-0.794264,0.837643
có,-0.459220,0.635046,1.350138,0.183445,-0.081955,1.386434,0.300067,1.478432,-0.044516,1.950213,...,-0.987272,-2.033924,-0.673144,-0.060245,1.192734,-1.099205,3.464785,1.503690,3.801477,0.025278
bạn,0.417472,-0.715568,1.517480,0.800126,-0.747393,-1.205578,0.423708,-1.424603,-0.854578,2.070962,...,-1.063569,-3.335137,2.059341,-1.481466,0.243240,-0.138777,2.984423,1.598934,3.053946,0.707018
gì,0.801491,-0.693707,-0.397485,0.013468,-1.085431,-0.669190,0.556483,-0.612736,-0.548655,2.155066,...,-1.088802,-2.438870,1.484580,1.622620,1.478590,-0.572223,3.279313,1.168636,3.528351,1.639949
đi,0.119332,-1.051064,-1.184523,1.337369,-0.875748,-1.443524,0.668860,-2.653937,-0.599493,3.014630,...,-1.327710,-0.418071,2.005838,1.765002,0.561932,0.326511,1.205037,1.421050,-2.172567,-0.012515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ruột,-0.069675,0.183172,-0.075308,-0.352660,0.316951,0.267559,-0.344167,0.253430,0.202614,-0.376453,...,0.294361,0.232183,-0.268291,-0.331619,-0.331592,-0.070576,-0.314324,-0.339956,-0.109202,-0.194799
nội ngoại,-0.238123,0.425722,-0.498322,-0.348847,0.380741,0.397564,-0.494957,0.422894,0.303767,-0.245704,...,0.522750,0.443184,-0.440309,-0.336196,-0.401741,-0.297569,-0.295356,-0.323639,-0.337634,-0.286470
bme,-0.441786,0.527095,-0.307128,-0.409318,0.349113,0.670163,-0.454369,0.300731,0.497565,-0.148799,...,0.476441,0.436414,-0.375764,-0.378880,-0.336960,-0.455281,-0.493796,-0.432233,-0.399628,-0.256421
vô duyên,-0.610372,0.716103,-0.646132,-0.608660,0.630604,0.623746,-0.590829,0.290480,0.559694,-0.395449,...,0.586203,0.605412,-0.564773,-0.475972,-0.622178,-0.428881,-0.583238,-0.650619,-0.524544,-0.611067


In [27]:
def compare(query):
  max_cs = 0
  max_id = 0
  query = remove_punctuation(query)
  query = query.lower()
  query = word_tokenize(query)

  query_arr = []
  for token in query:
    try:
      query_arr.append(list(df_embedding.loc[token]))
    except BaseException:
      pass 
  query_arr = np.array(query_arr)
  sum_query_arr = np.sum(query_arr,0)


  for i in range(0,len(sentences)):
    M = np.array([list(df_embedding.loc[token]) for token in sentences[i]])
    sum_arr = np.sum(M,0)

    #compare sentence(M) and query
    cs = 1 - spatial.distance.cosine(sum_query_arr, sum_arr)
    if cs > max_cs:
      max_cs = cs
      max_id = i
  
  return max_id

max_id = compare('Bạn học trường nào?')
df['user_b'][max_id]

ValueError: Input vector should be 1-D.

In [ ]:
print("**************************")
print("*    start chatting      *")
print("*    input q to exit     *")
print("**************************")
while True:
  newMessage  = input("you : ")
  if newMessage == 'q':
    print("chatbot :","tạm biệt")
    break
  max_id = compare(newMessage)
  print("chatbot :",df['user_b'][max_id])

**************************
*    start chatting      *
*    input q to exit     *
**************************
you : Bạn học trường nào
chatbot : Tớ học đại học Tôn Đức Thắng
you : Bạn bao nhiêu tuổi?
chatbot : mình 21 tuổi
you : Bạn bao nhiêu tuổi?
chatbot : mình 21 tuổi
you : Bạn có sở thích gì?
chatbot : Núi và sông.
you : có chơi thể thao không
chatbot : Có.
you : chiều đi đá banh không
chatbot : đi
you : ok
chatbot : chúc ngày mới vui vể
you : q
chatbot : tạm biệt


In [ ]:
# **************************
# *    start chatting      *
# *    input q to exit     *
# **************************
# you : Bạn học trường nào
# chatbot : Tớ học đại học Tôn Đức Thắng
# you : Bạn bao nhiêu tuổi?
# chatbot : mình 21 tuổi
# you : Bạn bao nhiêu tuổi?
# chatbot : mình 21 tuổi
# you : Bạn có sở thích gì?
# chatbot : Núi và sông.
# you : có chơi thể thao không
# chatbot : Có.
# you : chiều đi đá banh không
# chatbot : đi
# you : ok
# chatbot : chúc ngày mới vui vể
# you : q
# chatbot : tạm biệt